In [2]:
from typing import Tuple, List, Union, Any, Optional, Dict, Set, Literal, Callable
import os
import sys
from functools import partial
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
from jax.random import PRNGKey
import aeon
import pandas as pd

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils.utils import print_name, print_shape
from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero

jax.config.update('jax_platform_name', 'cpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

In [12]:
def _step_forward(
        Z: Float[Array, "N  d"],
        XTdiff: Float[Array, "N  D"],
        A: Float[Array, "d  d  D"],
        b: Float[Array, "d  D"],
        activation:Callable = jnp.tanh,
    ):
    """
    Inner loop of a Controlled ResNet.
    """
    Z = jnp.dot(Z, A) + b
    Z = activation(Z) * XTdiff[:, None, :]
    Z = Z.sum(axis=-1)
    return Z, None

N = 2
d = 3
D = 5

Z = jnp.ones((N, d))
A = jnp.ones((d, d, D))
b = jnp.ones((d, D))
XTdiff = jnp.ones((N, D))

_step_forward(Z, XTdiff, A, b)


(2, 3, 5)
(2, 3, 5)
(2, 3)


(Array([[4.997, 4.997, 4.997],
        [4.997, 4.997, 4.997]], dtype=float64),
 None)

In [13]:
seed = PRNGKey(0)
seed1, seed2 = jax.random.split(seed, 2)
seed3, seed4 = jax.random.split(seed2, 2)
print(seed1, seed2, seed3, seed4)
print(jax.random.split(seed, 4))
print(jax.random.split(seed, 3))

[4146024105  967050713] [2718843009 1272950319] [3746991216 1902760697] [1190051861 3378399878]
[[2285895361 1501764800]
 [1518642379 4090693311]
 [ 433833334 4221794875]
 [ 839183663 3740430601]]
[[2467461003  428148500]
 [3186719485 3840466878]
 [2562233961 1946702221]]
